<a href="https://colab.research.google.com/github/SknyL/Face_emotion_recognition/blob/main/Preparing_data_for_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загрузка датасета

In [ ]:
!gdown 1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16

In [ ]:
!unzip -q '/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/train.zip' -d '/content/data/'

Для детектирования лиц на изображении используем готовую библиотеку mediapipe

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 24.0 MB/s eta 0:00:00


In [ ]:
import mediapipe as mp
import os
import cv2
import time
import shutil

In [ ]:
def crop_img(img):
    """
      Изображение на входе должно быть RGB
    """
    # Создание менеджера контекста для детектора лиц
    with mp.solutions.face_detection.FaceDetection(
            model_selection=0, min_detection_confidence=0.7) as face_detection:
        # Поиск лица на кадре
        faces = face_detection.process(img)
        # Если лицо не обнаружено возвращается None
        if not faces.detections:
            return None
        for id, detection in enumerate(faces.detections):
            # Координаты и размеры бокса с лицом на кадре
            bbox = detection.location_data.relative_bounding_box
            mul_h = img.shape[0]
            mul_w = img.shape[1]
            x = int(abs(bbox.xmin) * mul_w)
            y = int(abs(bbox.ymin) * mul_h)
            w = int(abs(bbox.width) * mul_w)
            h = int(abs(bbox.height) * mul_h)
            # Обрезка лица из кадра
            img_crop = img[y:y+h, x:x+w, :]
            return img_crop

In [ ]:
# Путь для подготовленных изображений
croped_path = '/content/data/croped_train/' 
# Проверка на существование папки
if os.path.exists(croped_path): 
    shutil.rmtree(croped_path)

for (root,dirs,files) in os.walk('/content/data/train', topdown=False):
    folder = root.split('/')[-1] # Выделение название папки из пути
    croped_path = f'/content/data/croped_train/{folder}'
    os.makedirs(croped_path) # Создание папки
    for file in [root + '/' + file for file in files]:
        img = cv2.imread(file, cv2.COLOR_BGR2RGB) # Чтение изображения из папки
        img_crop = crop_img(img) # Применение функции обрезки лица из изображения
        # Если лицо не было детектированно на изображении цикл начинается заново
        if img_crop is None:
            continue
        # Путь до полученного изображения
        file_path = os.path.join(croped_path , file.split('/')[-1])
        save_flag = cv2.imwrite(file_path, img_crop) # Запись изображения
        print(f'File:{file_path} saved: {save_flag}') # Вывод пути и флага записи